In [54]:
import pickle as pk 
import pandas as pd

In [56]:
file_p = '../../../../Cre/DroughtNet/tophat/0.singleend_cov.dicHD2array.p'
dicHD2array = pk.load( open( file_p, "rb" ) )

In [57]:
file_gff = '../../../../Cre/DroughtNet/PhytozomeV10_download/References/Esalsugineum/annotation/Esalsugineum_173_v1.0.gene.gff3'
df_gff   = pd.read_csv(file_gff,sep='\t',skiprows=2,header=None)

In [58]:
mask        = (df_gff[2] == 'gene')
df_gff_gene = df_gff[mask]

In [59]:
df_gff_gene['genename'] = df_gff_gene[8].apply(lambda x : x.split(';')[0].replace('ID=','').split('.')[0])

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [60]:
dicHD2array['scaffold_1'][10:100]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=uint64)

In [112]:
def get_cov(df):
    scaffold   = df[0] 
    #print df
    #print scaffold
    left = df[3]
    right = df[4]
    #print left,right
    array = dicHD2array[scaffold][int(left)-1:int(right)]
    return float(len(array[(array > 0)]))/float(right-left+1)
df_gff_gene['coverage'] = df_gff_gene.apply(get_cov,axis=1)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [117]:
def get_dep(df):
    scaffold   = df[0] 
    #print df
    #print scaffold
    left = df[3]
    right = df[4]
    #print left,right
    array = dicHD2array[scaffold][int(left)-1:int(right)]
    return sum(array)/float(right-left+1)
df_gff_gene['depth'] = df_gff_gene.apply(get_dep,axis=1)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [118]:
df_gff_gene_ix = df_gff_gene.set_index('genename')

In [119]:
df_gff_gene_ix[df_gff_gene_ix['coverage']>0].head()

,0,1,2,3,4,5,6,7,8,coverage,depth
genename,,,,,,,,,,,
Thhalv10024176m,scaffold_1,phytozomev10,gene,10215584,10239664,.,+,.,ID=Thhalv10024176m.g.v1.0;Name=Thhalv10024176m.g,1,267.635148
Thhalv10024177m,scaffold_1,phytozomev10,gene,1677535,1695178,.,+,.,ID=Thhalv10024177m.g.v1.0;Name=Thhalv10024177m.g,1,138.790864
Thhalv10024179m,scaffold_1,phytozomev10,gene,488357,503965,.,+,.,ID=Thhalv10024179m.g.v1.0;Name=Thhalv10024179m.g,1,228.589980
Thhalv10024180m,scaffold_1,phytozomev10,gene,3576033,3586074,.,-,.,ID=Thhalv10024180m.g.v1.0;Name=Thhalv10024180m.g,1,204.641605
Thhalv10024181m,scaffold_1,phytozomev10,gene,15769002,15781579,.,-,.,ID=Thhalv10024181m.g.v1.0;Name=Thhalv10024181m.g,1,49.504452


In [120]:
from StringIO import StringIO

test = StringIO('''Thhalv10018014m,2 bands (400bp and 600 bp)
Thhalv10019947m,No                        
Thhalv10018181m,1 band (1500 bp)
Thhalv10024568m,No                        
Thhalv10007339m,No                    
Thhalv10002002m,2 bands (400bp and 600 bp)
Thhalv10025531m,Yes  (expected size)
Thhalv10010790m,No                         
Thhalv10013526m,Yes (expected size)
Thhalv10006290m,Yes (expected size)
Thhalv10014963m,Yes (expected size)''')

In [121]:
df = pd.DataFrame.from_csv(test,sep=',',header=None)

In [122]:
df = df.reset_index()

In [125]:

df['cov'] = df[0].apply(lambda x : df_gff_gene_ix.loc[x]['coverage'])
df['dep'] = df[0].apply(lambda x : df_gff_gene_ix.loc[x]['depth'])

In [126]:
df.head()

,0,1,cov,dep
0,Thhalv10018014m,2 bands (400bp and 600 bp),1,176.115945
1,Thhalv10019947m,No,1,202.985005
2,Thhalv10018181m,1 band (1500 bp),1,71.664586
3,Thhalv10024568m,No,1,538.662128
4,Thhalv10007339m,No,1,54.006396


In [104]:
df_gff_gene_ix.loc['Thhalv10018014m']['coverage']

1.0

In [139]:
gene  = 'Thhalv10018014m'
df_gff_gene_ix.loc[gene]

0                                                 scaffold_9
1                                               phytozomev10
2                                                       gene
3                                                    4701572
4                                                    4710213
5                                                          .
6                                                          -
7                                                          .
8           ID=Thhalv10018014m.g.v1.0;Name=Thhalv10018014m.g
coverage                                                   1
depth                                                176.116
Name: Thhalv10018014m, dtype: object

In [143]:
sum(dicHD2array['scaffold_9'][int(df_gff_gene_ix.loc[gene][3])-1:int(df_gff_gene_ix.loc[gene][4])])/float(int(df_gff_gene_ix.loc[gene][4])-int(df_gff_gene_ix.loc[gene][3])+1)

176.11594538301318